In [3]:
#Import important libraries
import pandas as pd
import numpy as np
import time
from datetime import datetime
import datetime as dt

## Task 1: What is the total number of entries across the subway system for August 12, 2017? 


We import the data. In addition we create another variable named **ID** by merging the variables **unit** and **scp**. The new variable **ID** uniquely identifies turnstiles and will help us iterate across all turnstiles to perform our calculations.

In [4]:
#Import data for 2017
data_2017=pd.read_csv('2017.csv')

#Define ID variable
data_2017['ID']=data_2017.unit+" "+data_2017.scp




In [5]:
#Preview the first 5 rows of the 2017 data
data_2017.head()

,time,ca,unit,scp,station,linename,division,desc,entries,exits,ID
0,2017-01-01 00:00:00,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,5992418.0,2028543.0,R051 02-00-00
1,2017-01-01 00:00:00,A002,R051,02-00-01,59 ST,NQR456W,BMT,REGULAR,5475502.0,1204841.0,R051 02-00-01
2,2017-01-01 00:00:00,A002,R051,02-03-00,59 ST,NQR456W,BMT,REGULAR,842188.0,3132405.0,R051 02-03-00
3,2017-01-01 00:00:00,A002,R051,02-03-01,59 ST,NQR456W,BMT,REGULAR,123036.0,218034.0,R051 02-03-01
4,2017-01-01 00:00:00,A002,R051,02-03-02,59 ST,NQR456W,BMT,REGULAR,5210075.0,7206414.0,R051 02-03-02


Because **entries** and **exits** are reported as cumulative values we assume that there should not be a 0 entry or exit value. In addition if a turnstile has a faulty 0 entry or exit value we consider the other value (exit or entry) faulty too. Thus we remove observations with 0 entry or exit values.


In [6]:
data_2017=data_2017[(data_2017.entries!=0)|(data_2017.exits!=0)]

We transform the **time** variable to timestamp, in order to manipulate it easier.

In [7]:
data_2017.time=[datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in data_2017.time ]

We also create two other variables **TIME** and **DATE** and assign the time and date components of the variable **time**. These variables might be useful later on.

In [8]:
data_2017['DATE']=[x.date() for x in data_2017.time]
data_2017['TIME']=[x.time() for x in data_2017.time]

We create a dataframe with observations only from 1st of August 2017.

In [10]:
#Assign and store the required day to a new dataframe
data_12Aug2017=data_2017[data_2017.DATE==dt.date(2017,8,12)].reset_index(drop=True)

To calculate the total count of entries in 1st of August we make the following assumptions:

• A turnstile cannot have 0 entry or exit activity. Hence between periods extries and exits must increase for each turnstile.

• For each turnstile the maximum entry and exit values occur in the last observation (the latest in time). Similarly the minimum count occurs in the first observation (the earliest in time).

• For each turstile we calculate the entry count by subtracting the last with the first entry observation.

Any turnstile that does not fulfil the above conditions is **excluded** from the calculations.

In [11]:
count_entries=0
#iterate across all turnstiles
for names in data_12Aug2017.ID.unique().tolist():
    #create a new dataframe with the unique turnstile and sort values by time
    dummy_df=data_12Aug2017[data_12Aug2017.ID==names].sort_values(by='time').reset_index(drop=True)
    nrows=dummy_df.shape[0]
    # Set the required conditions
    if (dummy_df.entries.max()>dummy_df.entries.min() 
    and dummy_df.exits.max()>dummy_df.exits.min()
    and dummy_df.entries.iloc[0]==dummy_df.entries.min() 
    and dummy_df.entries.iloc[nrows-1]==dummy_df.entries.max()
    and dummy_df.exits.iloc[0]==dummy_df.exits.min() 
    and dummy_df.exits.iloc[nrows-1]==dummy_df.exits.max() ):
            tourn_entries=dummy_df.iloc[nrows-1].entries-dummy_df.iloc[0].entries
            count_entries+=tourn_entries

In [12]:
count_entries

2402303.0

Running the cell above we get that the total number of entries in 12th of August is 2,402,303.

## Task 2: If we define traffic as the sum of the entry & exit count, what turnstile had the most traffic on August 1, 2017? 

To answer Task 2 we use exactly the same assumptions as for Q1. For each turnstile similar to counting entries above we will count exits by taking the difference between the last and the first exit observation.


In [14]:
#We create a dictionary with key the turnstile ID and value the turnstile's traffic 
tourn_traffic={}
for names in data_12Aug2017.ID.unique().tolist():
    dummy_df=data_12Aug2017[data_12Aug2017.ID==names].sort_values(by='time').reset_index(drop=True)
    nrows=dummy_df.shape[0]
    #Set conditions
    if (dummy_df.entries.max()>dummy_df.entries.min() 
    and dummy_df.exits.max()>dummy_df.exits.min()
    and dummy_df.entries.iloc[0]==dummy_df.entries.min() 
    and dummy_df.entries.iloc[nrows-1]==dummy_df.entries.max()):
            tourn_entries=dummy_df.iloc[nrows-1].entries-dummy_df.iloc[0].entries
            tourn_exits=dummy_df.iloc[nrows-1].exits-dummy_df.iloc[0].exits
            tourn_traffic[names]=tourn_entries+tourn_exits

In [21]:
#Find the turnstile with max entry
max_value = max(tourn_traffic.values()); 
{(key,value) for key, value in tourn_traffic.items() if value == max_value}

{('R301 00-00-00', 5683.0)}

Running the 2 cells above we get that the turnstile with unit **R031** and scp  **00-00-00**  had the maximum traffic on August 12th 2017 with 5682 entries and exits.

## Task 3: Which station had the highest average number of traffic between midnight and 4am on Wednesdays in July 2017? 

A quick view on the calendar gives us the Wednesday dates in July 2017. These are 2017-7-5, 2017-7-12, 2017-7-19, 2017-7-26. Below we create a dataframe with observations from these dates and required time horizon (00:00:00-04:00:00)

In [24]:
data_WedJuly=data_2017[(data_2017.DATE==dt.date(2017,7,5))|(data_2017.DATE==dt.date(2017,7,12))|(data_2017.DATE==dt.date(2017,7,19))|(data_2017.DATE==dt.date(2017,7,26))].sort_values(by='time').reset_index(drop=True)
data_WedJuly=data_WedJuly[(data_WedJuly.TIME>=datetime.strptime("00:00:00","%H:%M:%S").time())&(data_WedJuly.TIME<=datetime.strptime("04:00:00","%H:%M:%S").time())].sort_values(by='time').reset_index(drop=True)                  

For counting entries and exits we will follow the same recipe as in the previous questions.

A station might not have observations in one of the Wednesdays (00:00:00-04:00:00). We will consider these as 0 traffic count. Thus to get averages we divide by 4 (the number of Fridays in July) and not by the number of days, where the station is active.

Finally for each station, a turnstile might not be active in one of the Wednesdays. We remove such observations.

In [30]:
#We create a dictionary with key the stations and value the stations' average number of entries
mean_traffic={}
#Iterate across stations
for stations in data_WedJuly.station.unique().tolist():
    count_traffic=0.
    df_stations=data_WedJuly[data_WedJuly.station==stations].sort_values(by='time').reset_index(drop=True)
    #Iterate across Fridays
    for day in df_stations.DATE.unique().tolist():
        df_stations_day=df_stations[df_stations.DATE==day].sort_values(by='time').reset_index(drop=True)
       #Iterate across turnstiles 
        for names in df_stations_day.ID.unique().tolist():
            df_stations_day_names=df_stations_day[df_stations_day.ID==names].sort_values(by='time').reset_index(drop=True)
            nrows=df_stations_day_names.shape[0]
            #Set conditions
            if (df_stations_day_names.entries.max()>df_stations_day_names.entries.min() 
            and df_stations_day_names.exits.max()>df_stations_day_names.exits.min()
            #Remove inactive turnstiles
            and nrows!=0
            and df_stations_day_names.entries.iloc[0]==df_stations_day_names.entries.min() 
            and df_stations_day_names.entries.iloc[nrows-1]==df_stations_day_names.entries.max()
            and df_stations_day_names.exits.iloc[0]==df_stations_day_names.exits.min() 
            and df_stations_day_names.exits.iloc[nrows-1]==df_stations_day_names.exits.max()):
                tourn_entries=df_stations_day_names.iloc[nrows-1].entries-df_stations_day_names.iloc[0].entries
                tourn_exits=df_stations_day_names.iloc[nrows-1].exits-df_stations_day_names.iloc[0].exits
                tourn_traffic=tourn_entries+tourn_exits
                count_traffic+=tourn_traffic
                mean_traffic[stations]=count_traffic/4
       
                
 

In [31]:
max_value = max(mean_traffic.values()); 

{(key,value) for key, value in mean_traffic.items() if value == max_value}

{('TIMES SQ-42 ST', 27050.0)}

Running the 2 cells above we get that the station **TIMES SQ-42 ST** has the highest average number of traffic (27,050) on Fridays July 2017 between 12 am and 4 am.

## Task 4: If we compare the month of July 2016 to the month of July 2017, what station saw the highest relative traffic growth? 

First we will perform the same operations as in Q1 for the 2016 data. Same conditions apply

In [25]:
data_2016=pd.read_csv('mta-turnstile-data/2016.csv')
data_2016['ID']=data_2016.unit+" "+data_2016.scp
data_2016=data_2016[(data_2016.entries!=0)|(data_2016.exits!=0)]

In [26]:
data_2016.time=[datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in data_2016.time]
data_2016['DATE']=[x.date() for x in data_2016.time]
data_2016['TIME']=[x.time() for x in data_2016.time]

We restrict the dataframes now to July

In [27]:
data_July2016=data_2016[data_2016.time.dt.month==7]
data_July2017=data_2017[data_2017.time.dt.month==7]

Doing a simple data exploration we see that the number of stations in July 2016 and July 2017 are not the same (371 and 376 respectively). This warns us that some stations are not active in both periods. 

Hence we will choose stations that are active in both July 2016 and July 2017. 

In [28]:
#define sets with the stations
station_set2016=set(data_July2016.station)
station_set2017=set(data_July2017.station)
#Pick the common stations
common_stations=station_set2016.intersection(station_set2017)

Running the cell above we get 368 common stations. 

In order to calculate the highest relative traffic growth, we will use the formula:

• (traffic_Jul17-traffic_Jul16)/traffic_Jul16.

The formula components traffic_Jul17 and traffic_Jul16 are calculated as in the previous questions.

Because we compare data that are 1 year apart we want to be very careful of any malfunction leading to very inflated entry or exit values (turnstile malfunction, reset, etc.). 

For this reason we add an extra condition that for each turnstile between consecutive observations the entry or exit activity can not be more than 10,000.


In [30]:
#We create a dictionary with key the stations and value the stations relative growth
station_dic={}
#iterate through common stations
for station in common_stations:
    total_traffic2016=0
    total_traffic2017=0
    dummy_df2016=data_July2016[data_July2016.station==station].sort_values(by='time').reset_index(drop=True)
    dummy_df2017=data_July2017[data_July2017.station==station].sort_values(by='time').reset_index(drop=True)
    #Iterate through  July 2016 tunrstiles
    for name in dummy_df2016.ID.unique().tolist():
        dummy_df1=dummy_df2016[dummy_df2016.ID==name].sort_values(by='time').reset_index(drop=True)
        nrows1=dummy_df1.shape[0]
        #Set conditions
        if (dummy_df1.entries.max()>dummy_df1.entries.min() 
        and dummy_df1.exits.max()>dummy_df1.exits.min()
        and all(np.abs(np.diff(list(dummy_df1.entries))) < 1e4)
        and all(np.abs(np.diff(list(dummy_df1.exits))) < 1e4)
        and dummy_df1.entries.iloc[0]==dummy_df1.entries.min()
        and dummy_df1.entries.iloc[nrows1-1]==dummy_df1.entries.max()
        and dummy_df1.exits.iloc[0]==dummy_df1.exits.min()
        and dummy_df1.exits.iloc[nrows1-1]==dummy_df1.exits.max()):
            tourn_entries2016=dummy_df1.iloc[nrows1-1].entries-dummy_df1.iloc[0].entries
            tourn_exits2016=dummy_df1.iloc[nrows1-1].exits-dummy_df1.iloc[0].exits
            total_traffic2016+=tourn_entries2016+tourn_exits2016
        
        
        #Iterate through  July 2017 tunrstiles  
    for name1 in dummy_df2017.ID.unique().tolist():
        dummy_df2=dummy_df2017[dummy_df2017.ID==name1].sort_values(by='time').reset_index(drop=True)
        nrows2=dummy_df2.shape[0]
        #Set Conditions
        if (dummy_df2.entries.max()>dummy_df2.entries.min() 
        and dummy_df2.exits.max()>dummy_df2.exits.min()
        and all(np.abs(np.diff(list(dummy_df2.entries))) < 1e4)
        and all(np.abs(np.diff(list(dummy_df2.exits))) < 1e4)
        and dummy_df2.entries.iloc[0]==dummy_df2.entries.min()
        and dummy_df2.entries.iloc[nrows2-1]==dummy_df2.entries.max()
        and dummy_df2.exits.iloc[0]==dummy_df2.exits.min()
        and dummy_df2.exits.iloc[nrows2-1]==dummy_df2.exits.max()):
            tourn_entries2017=dummy_df2.iloc[nrows2-1].entries-dummy_df2.iloc[0].entries
            tourn_exits2017=dummy_df2.iloc[nrows2-1].exits-dummy_df2.iloc[0].exits
            total_traffic2017+=tourn_entries2017+tourn_exits2017
            if (total_traffic2016!=0 and total_traffic2017!=0):
                station_dic[station]=(total_traffic2017-total_traffic2016)/total_traffic2016

In [32]:
max_value = max(station_dic.values()); 
{key for key, value in station_dic.items() if value == max_value}




{'ST. GEORGE'}

Running the two cells above we get that **ST. GEORGE** station had the highest relative growth. The growth was 231%.

## Task 5: From July 2016 through July 2017, what date had the least traffic? 

First we create a joint dataframe for the 2016 and 2017 data.

In [33]:
data_2016_2017=data_2016.append(data_2017).sort_values(by='time').reset_index(drop=True)

We then restrict to the dates required, ie. from 2016/7/1 to 2017/7/31.

In [34]:
data_Jul_1617=data_2016_2017[(data_2016_2017.DATE>=datetime.strptime("2016-7-1", "%Y-%m-%d").date())& (data_2016_2017.DATE<=datetime.strptime("2017-7-31", "%Y-%m-%d").date())]

We will use the same conditions as in the previous questions for the calculations.

In [ ]:
#We create a dictionary with key the dates and value the dates' traffic
date_min={}
#iterate across dates
for date in data_Jul_1617.DATE.unique().tolist():
    total_count=0
    data_Jul_1617_date=data_Jul_1617[data_Jul_1617.DATE==date].sort_values(by='time').reset_index(drop=True)
    #Iterate across turnstiles
    for name in data_Jul_1617_date.ID.unique().tolist():
        data_Jul_1617_date_name=data_Jul_1617_date[data_Jul_1617_date.ID==name].sort_values(by='time').reset_index(drop=True)
        nrows=data_Jul_1617_date_name.shape[0]
        #Set conditons
        if (data_Jul_1617_date_name.entries.max()>data_Jul_1617_date_name.entries.min() 
        and data_Jul_1617_date_name.exits.max()>data_Jul_1617_date_name.exits.min()
        and data_Jul_1617_date_name.entries.iloc[0]==data_Jul_1617_date_name.entries.min() 
        and data_Jul_1617_date_name.entries.iloc[nrows-1]==data_Jul_1617_date_name.entries.max()
        and data_Jul_1617_date_name.exits.iloc[0]==data_Jul_1617_date_name.exits.min() 
        and data_Jul_1617_date_name.exits.iloc[nrows-1]==data_Jul_1617_date_name.exits.max()) :
            tourn_entries=data_Jul_1617_date_name.entries.iloc[nrows-1]-data_Jul_1617_date_name.entries.iloc[0]
            tourn_exits=data_Jul_1617_date_name.exits.iloc[nrows-1]-data_Jul_1617_date_name.exits.iloc[0]
            total_count+=tourn_entries+tourn_exits
            date_min[str(date)]=total_count

        

In [ ]:
min_value = min(date_min.values()); 
{key for key, value in date_min.items() if value == min_value}

Running the two cells above we get that the date with least traffic between July 2016 and July 2017 was
**2016-12-25**. This result makes sense since its Christmas Day and people stay home with their families or travel to their families. 